Install packages 

In [ ]:
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain
!pip install PyPDF2
!pip install chromadb
!pip install faiss-cpu
!pip install langchain_google_genai
!pip install langchain sqlalchemy
!pip install python-dotenv

Importing libraries

In [1]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

c:\Users\a.canod\AppData\Local\anaconda3\envs\entorno_pruebas_acd\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configure Gemini API key

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

GEMINI_API_KEY=os.getenv('GEMINI_API_KEY')

In [3]:
import google.generativeai as genai


gemini_api_secret_name = GEMINI_API_KEY

GOOGLE_API_KEY=gemini_api_secret_name
genai.configure(api_key=GOOGLE_API_KEY)

Fijar directorio de tabajo

In [4]:
comision = os.getenv('COMISION')
alumno = os.getenv('ALUMNO')
archivo = os.getenv('ARCHIVO')
pdf = f'{comision}{alumno}{archivo}'

Definir funciones

In [5]:
google_api_key=GEMINI_API_KEY
chat_history=[]
def get_pdf_text(pdf):
    text=""
    pdf_reader= PdfReader(pdf)
    for page in pdf_reader.pages:
        text+= page.extract_text()
    return  text
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks


def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=google_api_key)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")


def get_conversational_chain():

    prompt_template = """

    Please answer in spanish \n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0, google_api_key=google_api_key)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=google_api_key)

    new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    response = chain(
        {"input_documents":docs, "question":user_question}
        , return_only_outputs=True)
    print(response['output_text'])

In [6]:
'''
user_question = """
Haz un resumen muy detallado del documento.
Incluye el objetivo.
Incluye la(s) hipótesis.
Evalua exhaustivamente si el objetivo e hipótesis se alinean con la base de datos presentada.
Si existen medidas calculadas usando DAX, por favor listalas, si alguna es errónea menciona como podría mejorarse y especifíca que hace cada una. Incluye una breve explicación sobre las funciones DAX usadas en su construcción.
Si existen transformaciones realizadas usando Power Query, detalla cuales fueron dichas transformaciones, a que tablas afectan y cual fue su objetivo.
Presenta las conclusiones y evalúa rigurosamente si responden al objetivo e hipótesis planteadas inicialmente.
"""
raw_text = get_pdf_text(pdf)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)
user_input(user_question)
'''

'\nuser_question = """\nHaz un resumen muy detallado del documento.\nIncluye el objetivo.\nIncluye la(s) hipótesis.\nEvalua exhaustivamente si el objetivo e hipótesis se alinean con la base de datos presentada.\nSi existen medidas calculadas usando DAX, por favor listalas, si alguna es errónea menciona como podría mejorarse y especifíca que hace cada una. Incluye una breve explicación sobre las funciones DAX usadas en su construcción.\nSi existen transformaciones realizadas usando Power Query, detalla cuales fueron dichas transformaciones, a que tablas afectan y cual fue su objetivo.\nPresenta las conclusiones y evalúa rigurosamente si responden al objetivo e hipótesis planteadas inicialmente.\n"""\nraw_text = get_pdf_text(pdf)\ntext_chunks = get_text_chunks(raw_text)\nget_vector_store(text_chunks)\nuser_input(user_question)\n'

In [7]:
user_question = """
El documento presenta todas las tablas de una base de datos, sus campos (identificando si se trata de primary o foregigns keys) y las relaciones con las demás tablas.

Construye una consulta SQL compleja incluyendo unicamente las tablas: claim, activityfact, patientinsurance, payer, department, provider, patient, visit y visitbillvisit
y explícame cual es su funcionalidad en términos del negocio.
"""
raw_text = get_pdf_text(pdf)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)
user_input(user_question)

FileNotFoundError: [Errno 2] No such file or directory: 'NoneNoneNone'